Referências: 1º https://edisciplinas.usp.br/pluginfile.php/6909091/mod_resource/content/0/Lab2%20-%20Computa%C3%A7%C3%A3o%20Simb%C3%B3lica.pdf

In [ ]:
import sympy as sp

deltaL,Lo,deltaT,coef_linear = sp.symbols('deltaL,Lo,deltaT,coef_linear')


# Defina a expressão para alpha
alpha_expr = deltaL / (Lo * deltaT)


# Calcule a primeira derivada parcial de alpha em relação a DeltaL
derivada_parcial_deltaL = sp.diff(alpha_expr, deltaL)

# Exiba o resultado
print(derivada_parcial_deltaL)

1/(Lo*deltaT)


In [ ]:
import sympy as sp

# Defina as variáveis simbólicas
deltaL, Lo, deltaT, coef_linear = sp.symbols('deltaL Lo deltaT coef_linear')

# Crie uma expressão matemática com base nas variáveis simbólicas
alpha_expr = deltaL / (Lo * deltaT)

# Substitua os valores numéricos na expressão
alpha_value = alpha_expr.subs({deltaL: 0.6, Lo: 489, deltaT: 33})

# Exiba o resultado numérico
print(alpha_value)

# Calcule a primeira derivada parcial de alpha em relação a DeltaL
derivada_parcial_deltaL = sp.diff(alpha_expr, deltaL)
derivada_parcial_2 = derivada_parcial_deltaL.subs({alpha_expr:2,deltaL:23})

# Exiba o resultado
print(derivada_parcial_2)


3.71816322736568e-5
1/(Lo*deltaT)


In [ ]:
import sympy as sp

# Defina as variáveis simbólicas
deltaL, Lo, deltaT, alpha = sp.symbols('deltaL Lo deltaT alpha')

# Defina a expressão para alpha
alpha_expr = deltaL / (Lo * deltaT)

# Calcule a primeira derivada parcial de alpha em relação a DeltaL
derivada_parcial_deltaL = sp.diff(alpha_expr, deltaL)

# Avalie a derivada no ponto específico
valor_da_derivada_em_ponto = derivada_parcial_deltaL.subs({deltaL: 0.6, Lo: 489, deltaT: 33})

# Exiba o resultado
print(valor_da_derivada_em_ponto)


1/16137


In [ ]:
import sympy as sp

# Defina as variáveis simbólicas
x, y, z = sp.symbols('x y z')

# Defina a expressão
expr = x**2 + y**2 + z**2

# Crie uma lista de substituições (pares variável -> valor)
substituicoes = [(x, 2), (y, 3), (z, 4)]

# Use a função map para aplicar as substituições à expressão
expr_substituida = expr.subs(substituicoes)

# Exiba o resultado
print(expr_substituida)


29
